In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from zipfile import ZipFile
from typing import Dict, List
import random
import os
from tempfile import TemporaryDirectory
from create_submission import create_submission

# Read the training dataset


In [ ]:
train_data = pd.read_json('labeled_validation_dataset.jsonl',lines=True)
train_data.head()

,example_id,paragraph,summary
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


# Preprocessing

In [ ]:
# Data Preprocessing
# import re
# import string
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# nltk.download('stopwords')
# nltk.download('punkt')

# def clean_text(text):
#     # Remove URL links
#     text = re.sub(r'http\S+', '', text)

#     # Remove special characters and punctuation
#     text = ''.join(c for c in text if c not in string.punctuation)

#     # Convert to lowercase
#     text = text.lower()

#     # Tokenize text
#     tokens = word_tokenize(text)

#     # Remove stopwords
#     stop_words = set(stopwords.words('arabic'))
#     tokens = [token for token in tokens if token not in stop_words]

#     # Remove numbers
#     tokens = [token for token in tokens if not token.isnumeric()]

#     # Remove short words
#     tokens = [token for token in tokens if len(token) > 1]

#     # Join tokens back to form text
#     cleaned_text = ' '.join(tokens)

#     return cleaned_text
train_data['cleaned_paragraph'] = train_data['paragraph'].apply(lambda x: x.replace('/', '').replace('\\', '').replace('\n', '').replace('"', '').replace('...','').replace('..','').replace(':','').replace('»','').replace('«','').replace('●','').replace('•','').replace('font=&quot]',''))
train_data['cleaned_summary'] = train_data['summary'].apply(lambda x: x.replace('/', '').replace('\\', '').replace('\n', '').replace('"', '').replace('...','').replace('..','').replace(':','').replace('»','').replace('«','').replace('●','').replace('•','').replace('font=&quot]',''))

# train_data['cleaned_paragraph'] = train_data['paragraph'].apply(clean_text)
# train_data['cleaned_summary'] = train_data['summary'].apply(clean_text)
train_data.head()

,example_id,paragraph,summary,cleaned_paragraph,cleaned_summary
0,0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...,السوريون يصرون على استقلال بلادهم و مثلما رفض...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


# Load the Pretrained Transformer Model (mT5) on GPU


In [ ]:
model_name = "eslamxm/mt5-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Training code (Fine-tuning)


In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 60

for epoch in range(num_epochs):
    total_loss = 0

    for index, row in train_data.iterrows():
        input_text = row['cleaned_paragraph']
        target_summary = row['cleaned_summary']

        # Encoding
        input_tokens = tokenizer.encode(input_text, truncation=True, padding=True, return_tensors="pt").to(device)
        target_tokens = tokenizer.encode(target_summary, truncation=True, padding=True, return_tensors="pt").to(device)

        # Forward pass
        output = model(input_tokens, labels=target_tokens)

        # Compute loss
        loss = output.loss
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_data)

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch 1/60, Average Loss: 2.486291108967422
Epoch 2/60, Average Loss: 2.1577017763992408
Epoch 3/60, Average Loss: 1.9940348050811074
Epoch 4/60, Average Loss: 1.860260881386794
Epoch 5/60, Average Loss: 1.7370523752330185
Epoch 6/60, Average Loss: 1.6176028522578152
Epoch 7/60, Average Loss: 1.4991686154495587
Epoch 8/60, Average Loss: 1.3780455573812707
Epoch 9/60, Average Loss: 1.2495740524360113
Epoch 10/60, Average Loss: 1.1103788898749785
Epoch 11/60, Average Loss: 0.9658055791220108
Epoch 12/60, Average Loss: 0.824006998790549
Epoch 13/60, Average Loss: 0.6953906394638024
Epoch 14/60, Average Loss: 0.5794871518170679
Epoch 15/60, Average Loss: 0.4667342265027684
Epoch 16/60, Average Loss: 0.36994852203053313
Epoch 17/60, Average Loss: 0.2911251336215185
Epoch 18/60, Average Loss: 0.24500874443245785
Epoch 19/60, Average Loss: 0.1972615078647996
Epoch 20/60, Average Loss: 0.1560299668775557
Epoch 21/60, Average Loss: 0.12505400021766105
Epoch 22/60, Average Loss: 0.10883393851653

# Save the trained model


In [ ]:
torch.save(model.state_dict(), 'fine_tuned_model.pth')
print("Fine-tuning finished")

Fine-tuning finished


# validation_data

In [ ]:
val_data = pd.read_json('validation_data.jsonl',lines=True)
val_data['cleaned_paragraph'] = val_data['paragraph'].apply(lambda x: x.replace('/', '').replace('\\', '').replace('\n', '').replace('"', '').replace('...','').replace('..','').replace(':','').replace('»','').replace('«','').replace('●','').replace('•','').replace('font=&quot]',''))

val_data.head()

In [ ]:
generated_summaries = []

for index, row in val_data.iterrows():
    input_text = row['cleaned_paragraph']
    input_tokens = tokenizer.encode(input_text, truncation=True, padding=True, return_tensors="pt").to(device)
    summary_ids = model.generate(input_tokens, num_beams=10, max_length=200, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

# Add generated summaries to the validation dataset
val_data['summary'] = generated_summaries

# Print the updated validation dataset
val_data.head()

,example_id,paragraph,cleaned_paragraph,summary
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...,فتح بيت المقدس ١٥هـ ٦٣٦ بملحق الروم واللصوص ول...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...,الفتوحات الإسلامية في عصر الخلفاء الراشدين .وب...,كتب عمر بن الخطابة أنه صلى في كنيسة، فأدركته ا...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...,فتوحات بلاد شمال إفريقيا .قام المسلمون بالتوجه...,فتح المسلمون بلاد شمال إفريقيا، وعقد صلحا أهله...
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...,ألقينا الضوء على أهم فتوحات بلاد شمال إفريقيا،...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...,عزيزي الطالب عزيزتي الطالبة بعد أن توقفت حركة...,بعد أن توقفت حركة الفتوحات الإسلامية بسبب فتنة...


To convert the summary column in the validation_dataset to a format suitable for the create_submission function

In [ ]:
submission_dictionary = { example_id : summary for _, example_id, summary in val_data[['example_id', 'summary']].itertuples() }
submission_dictionary

In [ ]:
base_keys = val_data.example_id.values.tolist()
len(base_keys)

call method to create submision

In [ ]:
create_submission(output_file_path = "/content/new_submission.zip", submission_dictionary = submission_dictionary, base_keys = base_keys)

Submission of predictions.jsonl as .zip saved at /content/new_submission.zip
